<a href="https://colab.research.google.com/github/B-rowser/B-rowser/blob/main/Umaro_Email_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome! If you are reading this you are likely working with Umaro and are looking to send marketing emails to our clients. This google collab has been created as a place to remotely handle this work! This document has been created by Timofey Arapov please reach out to tarapov@umarofoods.com at anytime to get help with using this application.

This code is designed to automate the process of creating personalized email drafts for multiple recipients. It fetches data from a Google Sheet, processes the data, and then uses the information to generate customized email drafts.



1. The script reads email body templates from text files and stores them.
2. It fetches data from a specific Google Sheet.
3. The fetched data is processed into a list containing information like recipient name, email, and other details.
4. It generates personalized email drafts for each recipient by replacing placeholders in the email body templates with the recipient's information.
5. Finally, the script saves the created email drafts to the sender's Gmail account.


---


When you run this script, you'll need to provide access to your Google account so it can fetch data from the Google Sheet and save email drafts to your Gmail account. The link to the Google Sheet being used is provided at the end of the script.

To use this you will need your two things:
1. Your email text bodies - these are templates written to fit specific restuarants. Create the folder and upload the files on the left
2. The client_secret.json file this is provided to you by me. Think of it a password that lets you run this script.
3. Email addresses and menu items that come from the google spread sheet.


The google sheet is linked below.

# https://docs.google.com/spreadsheets/d/1lUZtcSylCR4KR0yd7eFNxfY1qEJcBeqYunkoxPhgQM8/edit#gid=0&fvid=613069593

What do the fields mean?



*   Name - Restaurant Name
*   Email - Email to which to send, can contain multiple emails but must be seperated with a space and comma.
* menu1- Menu Item 1 From Restaurant
* menu2- Menu Item 2 From Restaurant
* type - the template for the email body to use
* subject - the email subject line 
* ignore - A flag that tells the script to ignore these line **Important to not create drafts for emails already created**






In [6]:
%%capture

!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
!pip install Flask
!pip install pyngrok

In [3]:
%%capture 
%%bash

echo """
authtoken: 2PZvdC1vAQO24JBGLFgD4dpPNVK_27TdPyi92C5K8Drh5Em3F
version: 2
tunnels:
  default:
    proto: http
    hostname: tumaro.ngrok.app
    addr: 127.0.0.1:80

""" > /content/ngrok.yml

In [8]:
%%capture 

import os
import threading
import socket
import time
import os
import base64
import pickle
import google.auth
from pyngrok import ngrok, conf
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.exceptions import RefreshError
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.colab import auth
from pathlib import Path
from google.oauth2.credentials import Credentials as GoogleCredentials
from http.server import BaseHTTPRequestHandler, HTTPServer
from flask import Flask, request, redirect
from google.oauth2 import service_account


conf.get_default().config_path = "/content/ngrok.yml"

def find_open_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(("", 0))
        return s.getsockname()[1]

os.environ["FLASK_DEBUG"] = "development"

app = Flask(__name__)

port = find_open_port()

# Pass custom_config to ngrok.connect()
public_url = ngrok.connect(port, subdomain = "tumaroapp" ).public_url



print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL

@app.route('/')
def index():
    return "Hello from Flask!"


@app.route('/authorize')
def authorize():
    auth_code = request.args.get('code')
    if auth_code:
        # You can now use this auth_code in your InstalledAppFlow
        flow = InstalledAppFlow.from_client_secrets_file(
            'client_secrets.json', 
            scopes=[
                'https://www.googleapis.com/auth/gmail.compose', 
                'https://www.googleapis.com/auth/spreadsheets.readonly', 
                'https://www.googleapis.com/auth/gmail.modify'
            ],
            redirect_uri=public_url+"/authorize"
        )
        flow.fetch_token(code=auth_code)
        creds = flow.credentials
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
        # Continue with your application logic here using the creds
        return "Authorization successful"
    else:
        return "No authorization code found. Please check the URL."

        
# Start the Flask server in a new thread
  server_thread = threading.Thread(target=app.run, kwargs={"host": "127.0.0.1", "port": port, "use_reloader": False}).start()
  server_thread.start()
  shutdown_thread = threading.Thread(target=shutdown_flask_app)
  shutdown_thread.start()




 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:34035
INFO:werkzeug:Press CTRL+C to quit
